In [23]:
import parselmouth
import glob
import os
import statistics
import numpy as np
import pandas as pd
import scipy.io as sio
from typing import Dict, Iterator, Iterable
from parselmouth.praat import call

In [2]:
# Get f0min and f0max
f0min_male = 80
f0max_male = 250
f0min_female = 140
f0max_female = 310

In [21]:
# Get formants (from PraatScripts by https://github.com/drfeinberg/PraatScripts/, modified to remove median)
def measureFormants(file, f0min, f0max):
    sound = parselmouth.Sound(file) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    try:
        f1_mean = statistics.mean(f1_list)
        f2_mean = statistics.mean(f2_list)
        f3_mean = statistics.mean(f3_list)
        f4_mean = statistics.mean(f4_list)
    except statistics.StatisticsError:
        print("Skipping: " + file + ", no formants extracted")
        return None
    
    return f1_mean, f2_mean, f3_mean, f4_mean

# (F1_actual + F1_feedback)/F1_actual
def getPerturbationValues(actual, feedback, f0min, f0max):
    perturbation_values: Dict[str, float] = {
        "f1": [],
        "f2": [],
        "f3": [],
        "f4": [],
    }
    
    first_trial = True
    
    for i in range(1, len(actual)):
        print("Fetching formants from " + actual[i])
        
        # Get formants
        actual_formants = measureFormants(actual[i], f0min, f0max)
        feedback_formants = measureFormants(feedback[i], f0min, f0max)
        
        if actual_formants is None or feedback_formants is None:
            continue
        else:
            (f1_actual, f2_actual, f3_actual, f4_actual) = actual_formants
            (f1_feedback, f2_feedback, f3_feedback, f4_feedback) = feedback_formants
            f1_perturb = (f1_actual + f1_feedback)/f1_actual
            f2_perturb = (f2_actual + f2_feedback)/f2_actual
            f3_perturb = (f3_actual + f3_feedback)/f3_actual
            f4_perturb = (f4_actual + f4_feedback)/f4_actual
        
        perturbation_values["f1"].append(f1_perturb)
        perturbation_values["f2"].append(f2_perturb)
        perturbation_values["f3"].append(f3_perturb)
        perturbation_values["f4"].append(f4_perturb)
        
    perturbation_values = pd.DataFrame(perturbation_values)
    return perturbation_values

def measurePitch(file):
    sound = parselmouth.Sound(file)
    pitch = sound.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    return pitch_values

def getPitchOfEachSyllable(pitches: Iterable[float]) -> Iterator[float]:                                                     
    acc = 0.0                                                                                                                 
    count = 0                                                                                                                 
    for pitch in pitches:                                                                                                     
        if pitch:                                                                                                             
            # We're in the start/middle of a syllable                                                                         
            acc += pitch                                                                                                      
            count += 1                                                                                                        
        elif count:                                                                                                           
            # We're at the end of a syllable                                                                                  
            yield acc / count                                                                                                 
            acc = 0                                                                                                           
            count = 0                                                                                                         
        else:                                                                                                                 
            # We're between syllables                                                                                         
            pass    
    if count:
        yield acc / count
        
def getPitchForOneStim(file):
#     sound = parselmouth(file)
    
    # Get pitches across utterance
    pitch_values = measurePitch(file)
    
    # Separate pitches for each syllable
    syllable_pitches = list(getPitchOfEachSyllable(pitch_values))
    
    return syllable_pitches
        
def getPitchForAllStim(stim_path):
    stim = glob.glob(stim_path)
    stim_pitches = {}

    for file in stim:
        syllable_pitches = getPitchForOneStim(file)
        
        # Save in dict
        stim_pitches[file.split('/')[-1]] = syllable_pitches
        
    return stim_pitches

In [24]:
os.chdir('/Users/letitiaho/src/talker_change_data_processing')
stim_pitches = getPitchForAllStim('0_set_up_and_raw_data/data/stim/low_pass_400/*.wav')
sio.savemat('7_coherence/data/syllable_pitches.mat', stim_pitches)